In [ ]:
# 🚀 世界クラス拡張メトリクス実装
import scipy.stats as stats
from scipy import stats
import pandas as pd
import numpy as np

def calculate_world_class_metrics(portfolio_values, returns, initial_capital, risk_free_rate=0.02):
    """世界クラス投資戦略用の拡張メトリクス計算"""
    
    if len(portfolio_values) == 0 or len(returns) == 0:
        return {}
    
    portfolio_series = pd.Series(portfolio_values)
    returns_series = pd.Series(returns).dropna()
    
    # 基本パフォーマンス指標
    total_return = (portfolio_series.iloc[-1] / initial_capital) - 1
    annual_return = (1 + total_return) ** (252 / len(portfolio_series)) - 1
    volatility = returns_series.std() * np.sqrt(252)
    
    # Sharpe Ratio (世界クラス目標: > 3.0)
    excess_return = annual_return - risk_free_rate
    sharpe_ratio = excess_return / volatility if volatility > 0 else 0
    
    # 最大ドローダウン (世界クラス目標: < 5%)
    cumulative = portfolio_series / portfolio_series.expanding().max()
    drawdown = (cumulative - 1) * 100  # パーセント表示
    max_drawdown = drawdown.min()
    
    # Calmar Ratio (世界クラス目標: > 0.6)
    calmar_ratio = annual_return / abs(max_drawdown / 100) if max_drawdown != 0 else 0
    
    # VaR (Value at Risk) 95% (世界クラス目標: < 2%)
    var_95 = np.percentile(returns_series, 5) * 100  # 95% VaR (5th percentile)
    daily_var_95 = abs(var_95)
    
    # CVaR (Conditional VaR) - Expected Shortfall
    var_threshold = np.percentile(returns_series, 5)
    cvar_95 = returns_series[returns_series <= var_threshold].mean() * 100
    
    # Sortino Ratio (下方偏差ベース)
    downside_returns = returns_series[returns_series < 0]
    downside_volatility = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0
    sortino_ratio = excess_return / downside_volatility if downside_volatility > 0 else 0
    
    # Maximum Drawdown Duration (回復期間)
    drawdown_duration = 0
    peak_idx = 0
    max_duration = 0
    
    for i, dd in enumerate(drawdown):
        if dd == 0:  # 新高値更新
            duration = i - peak_idx
            max_duration = max(max_duration, duration)
            peak_idx = i
        
    max_dd_duration = max_duration
    
    # Omega Ratio (利益の非対称性)
    threshold = 0
    gains = returns_series[returns_series > threshold]
    losses = returns_series[returns_series <= threshold]
    omega_ratio = gains.sum() / abs(losses.sum()) if len(losses) > 0 and losses.sum() != 0 else np.inf
    
    # Tail Ratio
    top_10_pct = np.percentile(returns_series, 90)
    bottom_10_pct = np.percentile(returns_series, 10)
    tail_ratio = abs(top_10_pct / bottom_10_pct) if bottom_10_pct != 0 else np.inf
    
    # Stability Metrics
    monthly_returns = returns_series.resample('M').apply(lambda x: (1 + x).prod() - 1)
    consistency = len(monthly_returns[monthly_returns > 0]) / len(monthly_returns) if len(monthly_returns) > 0 else 0
    
    # Risk-Adjusted Returns
    risk_adjusted_return = annual_return / max(volatility, 0.01)  # Return per unit risk
    
    # Win/Loss Ratio Analysis
    positive_returns = returns_series[returns_series > 0]
    negative_returns = returns_series[returns_series < 0]
    
    win_rate = len(positive_returns) / len(returns_series) if len(returns_series) > 0 else 0
    avg_win = positive_returns.mean() if len(positive_returns) > 0 else 0
    avg_loss = negative_returns.mean() if len(negative_returns) > 0 else 0
    win_loss_ratio = abs(avg_win / avg_loss) if avg_loss != 0 else np.inf
    
    # Kelly Criterion (最適ポジションサイズ)
    if win_rate > 0 and avg_loss != 0:
        kelly_f = win_rate - ((1 - win_rate) / abs(avg_win / avg_loss))
    else:
        kelly_f = 0
    
    return {
        # 基本指標
        'total_return': total_return,
        'annual_return': annual_return,
        'volatility': volatility,
        
        # 世界クラス必須指標
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown_pct': max_drawdown,
        'calmar_ratio': calmar_ratio,
        'daily_var_95': daily_var_95,
        
        # 拡張リスク指標
        'cvar_95': abs(cvar_95),
        'sortino_ratio': sortino_ratio,
        'max_dd_duration_days': max_dd_duration,
        'omega_ratio': omega_ratio,
        'tail_ratio': tail_ratio,
        
        # パフォーマンス指標
        'win_rate': win_rate,
        'avg_win': avg_win,
        'avg_loss': avg_loss,
        'win_loss_ratio': win_loss_ratio,
        'consistency': consistency,
        'risk_adjusted_return': risk_adjusted_return,
        'kelly_f': kelly_f,
        
        # 最終評価
        'final_capital': portfolio_series.iloc[-1],
        'total_trades': len(returns_series),
    }

def check_world_class_criteria(metrics):
    """世界クラス基準の達成度をチェック"""
    criteria = {
        'sharpe_ratio': {'target': 3.0, 'achieved': metrics.get('sharpe_ratio', 0) >= 3.0},
        'max_drawdown': {'target': -5.0, 'achieved': metrics.get('max_drawdown_pct', -100) >= -5.0},
        'calmar_ratio': {'target': 0.6, 'achieved': metrics.get('calmar_ratio', 0) >= 0.6},
        'daily_var_95': {'target': 2.0, 'achieved': metrics.get('daily_var_95', 100) <= 2.0},
        'win_rate': {'target': 0.55, 'achieved': metrics.get('win_rate', 0) >= 0.55},
    }
    
    total_achieved = sum(1 for c in criteria.values() if c['achieved'])
    total_criteria = len(criteria)
    achievement_rate = total_achieved / total_criteria
    
    return criteria, total_achieved, total_criteria, achievement_rate

def display_world_class_results(metrics):
    """世界クラス結果の美しい表示"""
    print(f"\n{'='*80}")
    print("🏆 WORLD-CLASS BACKTEST RESULTS")
    print(f"{'='*80}")
    
    # 核心指標
    print(f"\n🎯 Core Performance Metrics:")
    print(f"├─ Total Return:      {metrics.get('total_return', 0):8.2%}")
    print(f"├─ Annual Return:     {metrics.get('annual_return', 0):8.2%}")
    print(f"├─ Volatility:        {metrics.get('volatility', 0):8.2%}")
    print(f"└─ Final Capital:     ${metrics.get('final_capital', 0):,.2f}")
    
    # 世界クラス指標
    print(f"\n🚀 World-Class Criteria:")
    sharpe = metrics.get('sharpe_ratio', 0)
    max_dd = metrics.get('max_drawdown_pct', -100)
    calmar = metrics.get('calmar_ratio', 0)
    var_95 = metrics.get('daily_var_95', 100)
    win_rate = metrics.get('win_rate', 0)
    
    print(f"├─ Sharpe Ratio:      {sharpe:8.3f} {'✅' if sharpe >= 3.0 else '❌'} (Target: ≥3.0)")
    print(f"├─ Max Drawdown:      {max_dd:8.2f}% {'✅' if max_dd >= -5.0 else '❌'} (Target: ≥-5%)")
    print(f"├─ Calmar Ratio:      {calmar:8.3f} {'✅' if calmar >= 0.6 else '❌'} (Target: ≥0.6)")
    print(f"├─ Daily VaR95:       {var_95:8.2f}% {'✅' if var_95 <= 2.0 else '❌'} (Target: ≤2%)")
    print(f"└─ Win Rate:          {win_rate:8.2%} {'✅' if win_rate >= 0.55 else '❌'} (Target: ≥55%)")
    
    # 基準達成度
    criteria, achieved, total, rate = check_world_class_criteria(metrics)
    print(f"\n🏅 Achievement Score: {achieved}/{total} ({rate:.1%})")
    
    if rate >= 0.8:
        print("🎉 WORLD-CLASS PERFORMANCE ACHIEVED!")
    elif rate >= 0.6:
        print("💪 Strong Performance - Near World-Class!")
    else:
        print("⚠️ Performance needs improvement for world-class status")
    
    # 拡張リスク指標
    print(f"\n📊 Advanced Risk Metrics:")
    print(f"├─ Sortino Ratio:     {metrics.get('sortino_ratio', 0):8.3f}")
    print(f"├─ CVaR 95%:          {metrics.get('cvar_95', 0):8.2f}%")
    print(f"├─ Omega Ratio:       {metrics.get('omega_ratio', 0):8.3f}")
    print(f"├─ Win/Loss Ratio:    {metrics.get('win_loss_ratio', 0):8.3f}")
    print(f"└─ Consistency:       {metrics.get('consistency', 0):8.2%}")
    
    return criteria, achieved, total, rate

print("✅ 世界クラス拡張メトリクス実装完了")


In [ ]:
%%capture setup_output
# Google Drive マウント & 必要ライブラリのインストール
try:
    from google.colab import drive
    drive.mount("/content/drive")
    
    # パスの設定
    import sys
    sys.path.append("/content/drive/MyDrive/kucoin_bot")
    
    # 必要なライブラリをインストール
    %pip install -q scikit-learn lightgbm xgboost tensorflow pandas_ta pyti ta
    %pip install -q plotly ipywidgets
    
    print("✅ Google Colab セットアップ完了！")
    COLAB_ENV = True
except ImportError:
    print("ローカル環境で実行中")
    COLAB_ENV = False


In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import os
import pickle
import warnings
warnings.filterwarnings('ignore')

# 可視化設定
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['font.size'] = 10

# プロジェクト専用ライブラリ
try:
    from utils.drive_io import (
        save_data, 
        load_data, 
        ModelUtils,
        DataPreprocessor,
        BacktestUtils,
        get_project_path
    )
    print("✅ プロジェクト専用ライブラリ読み込み完了")
except ImportError as e:
    print(f"⚠️ プロジェクト専用ライブラリの読み込みに失敗: {e}")
    print("基本ライブラリのみで続行します")

print("✅ ライブラリのインポート完了")


In [ ]:
# バックテスト設定
CONFIG = {
    "symbol": "SOL_USDT",
    "initial_capital": 10000,  # 初期資金（USDT）
    "transaction_cost": 0.001,  # 取引手数料（0.1%）
    "min_confidence": 0.6,  # 最小信頼度閾値
    
    # リスク管理
    "stop_loss": 0.05,  # ストップロス（5%）
    "take_profit": 0.10,  # テイクプロフィット（10%）
    "max_position_size": 0.2,  # 最大ポジションサイズ（20%）
    
    # ファイル設定
    "model_date": datetime.now().strftime("%Y%m%d"),  # 今日の日付
    "reports_dir": "data/reports"
}

print(f"対象銘柄: {CONFIG['symbol']}")
print(f"初期資金: ${CONFIG['initial_capital']:,.2f}")
print(f"取引手数料: {CONFIG['transaction_cost']*100:.2f}%")


In [ ]:
class TradingStrategy:
    """トレーディング戦略基底クラス"""
    
    def __init__(self, initial_capital, transaction_cost, stop_loss=None, take_profit=None):
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        
        # 状態変数
        self.reset()
        
    def reset(self):
        """戦略状態をリセット"""
        self.capital = self.initial_capital
        self.position = 0  # 現在ポジション（正=ロング、負=ショート、0=無ポジション）
        self.position_value = 0  # ポジション評価額
        self.entry_price = 0  # エントリー価格
        self.trades = []  # 取引履歴
        self.portfolio_values = []  # ポートフォリオ価値推移
        
    def generate_signal(self, row):
        """シグナル生成（サブクラスで実装）"""
        raise NotImplementedError
        
    def execute_trade(self, price, signal, date):
        """取引実行"""
        if signal == 1 and self.position <= 0:  # ロングエントリー
            self._enter_long(price, date)
        elif signal == -1 and self.position >= 0:  # ショートエントリー
            self._enter_short(price, date)
        elif signal == 0:  # ポジション決済
            self._close_position(price, date)
            
    def _enter_long(self, price, date):
        """ロングポジション開始"""
        if self.position < 0:  # ショートポジションがある場合は決済
            self._close_position(price, date)
            
        # 利用可能資金でロングポジション
        available_capital = self.capital * (1 - self.transaction_cost)
        position_size = available_capital / price
        
        if position_size > 0:
            self.position = position_size
            self.entry_price = price
            self.capital = 0
            
            self.trades.append({
                'date': date,
                'action': 'LONG_ENTRY',
                'price': price,
                'quantity': position_size,
                'capital': self.capital + self.position * price
            })
            
    def _enter_short(self, price, date):
        """ショートポジション開始"""
        if self.position > 0:  # ロングポジションがある場合は決済
            self._close_position(price, date)
            
        # ショートポジション（資金の範囲内で）
        available_capital = self.capital * (1 - self.transaction_cost)
        position_size = -(available_capital / price)  # 負の値
        
        if position_size < 0:
            self.position = position_size
            self.entry_price = price
            self.capital = available_capital  # ショート時は資金は維持
            
            self.trades.append({
                'date': date,
                'action': 'SHORT_ENTRY',
                'price': price,
                'quantity': position_size,
                'capital': self.capital
            })
            
    def _close_position(self, price, date):
        """ポジション決済"""
        if self.position == 0:
            return
            
        if self.position > 0:  # ロング決済
            proceeds = self.position * price * (1 - self.transaction_cost)
            action = 'LONG_EXIT'
        else:  # ショート決済
            # ショート決済では差額を計算
            pnl = abs(self.position) * (self.entry_price - price)
            proceeds = self.capital + pnl * (1 - self.transaction_cost)
            action = 'SHORT_EXIT'
            
        self.capital = proceeds
        closed_quantity = self.position
        self.position = 0
        self.entry_price = 0
        
        self.trades.append({
            'date': date,
            'action': action,
            'price': price,
            'quantity': closed_quantity,
            'capital': self.capital
        })
        
    def check_stop_conditions(self, current_price, date):
        """ストップロス・テイクプロフィットのチェック"""
        if self.position == 0 or self.entry_price == 0:
            return
            
        if self.position > 0:  # ロングポジション
            price_change = (current_price - self.entry_price) / self.entry_price
        else:  # ショートポジション
            price_change = (self.entry_price - current_price) / self.entry_price
            
        # ストップロス
        if self.stop_loss and price_change <= -self.stop_loss:
            self._close_position(current_price, date)
            
        # テイクプロフィット
        elif self.take_profit and price_change >= self.take_profit:
            self._close_position(current_price, date)
            
    def update_portfolio_value(self, current_price):
        """ポートフォリオ価値の更新"""
        if self.position > 0:  # ロングポジション
            self.position_value = self.position * current_price
            current_portfolio_value = self.capital + self.position_value
        elif self.position < 0:  # ショートポジション
            pnl = abs(self.position) * (self.entry_price - current_price)
            current_portfolio_value = self.capital + pnl
        else:  # 無ポジション
            current_portfolio_value = self.capital
            
        self.portfolio_values.append(current_portfolio_value)
        return current_portfolio_value


class MLStrategy(TradingStrategy):
    """機械学習ベースの戦略"""
    
    def __init__(self, model, min_confidence=0.6, **kwargs):
        super().__init__(**kwargs)
        self.model = model
        self.min_confidence = min_confidence
        
    def generate_signal(self, row):
        """機械学習モデルからシグナル生成"""
        # 予測確率を取得
        prediction_prob = row.get('prediction_prob', 0.5)
        
        if prediction_prob > self.min_confidence:
            return 1  # ロングシグナル
        elif prediction_prob < (1 - self.min_confidence):
            return -1  # ショートシグナル
        else:
            return 0  # 無ポジション

print("✅ 戦略クラス実装完了")


In [ ]:
# 全ての出力を表示
print("🔍 セットアップ出力:")
print(setup_output)
print("\n" + "="*50)

print("\n✅ すべての処理が完了しました！")
print("📁 このノートブックは正常に動作する準備が整いました。")
print("\n次のステップ:")
print("1. モデル・データ読み込みセルを追加")
print("2. バックテスト実行セルを追加")
print("3. パフォーマンス計算・チャート作成セルを追加")
print("4. レポート保存セルを追加")
